In [31]:
import pandas as pd
import numpy as np
import openpyxl
from pymongo import MongoClient
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


In [32]:
# Open the xlsx file and convert to a dataframe
# Load spreadsheet
xl = pd.ExcelFile('MES.xlsx')
# Load a sheet into a DataFrame by its name
dfs = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

In [33]:
# Separate every element of the dictionary into a different dataframe
principales_cuentas = dfs['1']
balance_general = dfs['2']
estado_resultados = dfs['3']
ingresos_egresos = dfs['4']
detalle_ingresos = dfs['5']
detalle_egresos = dfs['6']
indicadores = dfs['7']

In [34]:
principales_cuentas_columns = principales_cuentas.columns.tolist()
balance_general_columns = balance_general.columns.tolist()
estado_resultados_columns = estado_resultados.columns.tolist()
ingresos_egresos_columns = ingresos_egresos.columns.tolist()
detalle_ingresos_columns = detalle_ingresos.columns.tolist()
detalle_egresos_columns = detalle_egresos.columns.tolist()
indicadores_columns = indicadores.columns.tolist()

In [35]:
rows = ["Total de Activos", "Total de Pasivos", "Capital Social", "Resultado del Ejercicio", "Total Patrimonio Neto", "Primas Directas", "Primas Reaseguros Aceptados - Local", 
        "Siniestros Seguros Directos", "Resultado Técnico Bruto [7]=[3]-[6]", "Gastos De Producción", "Gastos De Cesión Reaseguros - Local", "Gastos De Cesión Reaseguros - Exterior", 
        "Gastos Técnicos De Explotación", "Constitución De Previsiones", "Resultado Técnico Neto [11]=[7]+[10]", "Resultado Total del Ejercicio", "Resultado del Ejercicio"]
columns = ['El Comercio Paraguayo S.A. De Seguros', 'La Rural S.A. De Seguros', 'La Paraguaya S.A. De Seguros', 'Seguros Generales S. A. (Segesa)', 'Rumbos S.A. De Seguros', 
           'La Consolidada S.A. De Seguros', 'El Productor S.A. De Seguros Y Reaseguros', 'Atalaya S.A De Seguros Generales', 'La Independencia De Seguros Sociedad Anonima', 
           'Patria S.A. De Seguros Y Reaseguros', 'Alianza Garantía Seguros Y Reaseguros S.A.', 'Aseguradora Paraguaya S.A', 'Fénix S.A. De Seguros Y Reaseguros', 
           'Central S.A. De Seguros', 'Seguros Chaco S.A. De Seguros Y Reaseguros', 'El Sol Del Paraguay Compañía De Seguros Y Reaseguros', 'Intercontinental De Seguros Y Reaseguros S.A.', 
           'Seguridad S.A. Compañía De Seguros', 'Aseguradora Yacyreta S.A. De Seguros Y Reaseguros', 'La Agrícola S.A. De Seguros Y Reaseguros', 'Ueno Seguros S.A.', 'Cenit De Seguros S.A.', 
           'La Meridional Paraguaya S.A. De Seguros', 'Aseguradora Del Este S.A De Seguros Y Reaseguros', 'Regional S.A. De Seguros Y Reaseguros', 'Mapfre Paraguay Compañía De Seguros S.A.', 
           'Aseguradora Tajy Propiedad Cooperativa S.A. De Seguros', 'Panal Compañía De Seguros Generales S.A.', 'Sancor Seguros Del Paraguay S.A.', 'Royal Seguros S.A. Compañía De Seguros', 
           'Nobleza Seguros S.A. Compañia De Seguros', 'Itau Seguros Paraguay S.A.', 'Familiar Seguros S.A.', 'Atlas S.A. De Seguros', 'Total Mercado']
dataframes = [principales_cuentas, balance_general, estado_resultados, ingresos_egresos, detalle_ingresos, detalle_egresos, indicadores]
dataframes_names = ["principales_cuentas", "balance_general", "estado_resultados", "ingresos_egresos", "detalle_ingresos", "detalle_egresos", "indicadores"]
print(columns)

['El Comercio Paraguayo S.A. De Seguros', 'La Rural S.A. De Seguros', 'La Paraguaya S.A. De Seguros', 'Seguros Generales S. A. (Segesa)', 'Rumbos S.A. De Seguros', 'La Consolidada S.A. De Seguros', 'El Productor S.A. De Seguros Y Reaseguros', 'Atalaya S.A De Seguros Generales', 'La Independencia De Seguros Sociedad Anonima', 'Patria S.A. De Seguros Y Reaseguros', 'Alianza Garantía Seguros Y Reaseguros S.A.', 'Aseguradora Paraguaya S.A', 'Fénix S.A. De Seguros Y Reaseguros', 'Central S.A. De Seguros', 'Seguros Chaco S.A. De Seguros Y Reaseguros', 'El Sol Del Paraguay Compañía De Seguros Y Reaseguros', 'Intercontinental De Seguros Y Reaseguros S.A.', 'Seguridad S.A. Compañía De Seguros', 'Aseguradora Yacyreta S.A. De Seguros Y Reaseguros', 'La Agrícola S.A. De Seguros Y Reaseguros', 'Ueno Seguros S.A.', 'Cenit De Seguros S.A.', 'La Meridional Paraguaya S.A. De Seguros', 'Aseguradora Del Este S.A De Seguros Y Reaseguros', 'Regional S.A. De Seguros Y Reaseguros', 'Mapfre Paraguay Compañía 

In [36]:
all_equal = (balance_general_columns == estado_resultados_columns == 
             ingresos_egresos_columns == detalle_ingresos_columns == detalle_egresos_columns == indicadores_columns)

print("All column lists are equal:" , all_equal)

All column lists are equal: True


In [37]:
def get_value_from_df(df, row_value, column_name):
    try:
        # Find the row where 'Nombre de la Cuenta' is equal to row_value
        row = df[df['Nombre de la Cuenta'] == row_value]
        
        # Return the value at the intersection of the row and column_name
        return row[column_name].values[0]
    except (IndexError, KeyError):
        return "Error: Invalid row or column names. Please verify and try again."

In [38]:
def create_new_df(dfs, rows, columns):
    # Create a new dataframe with the first column as "Nombre de la Cuenta"
    new_df = pd.DataFrame(rows, columns=["Nombre de la Cuenta"])

    # Add new columns to the dataframe
    for column in columns:
        new_df[column] = None

    # Iterate over each dataframe
    for df in dfs:
        # Iterate over each row
        for i, row in enumerate(rows):
            # Iterate over each column
            for column in columns:
                # Get the value at the intersection of the row and column
                value = get_value_from_df(df, row, column)

                # If the value is not an error message, assign it to the new dataframe
                if not isinstance(value, str):
                    new_df.at[i, column] = value

    # Return the new dataframe
    return new_df

In [63]:
def create_new_dfs(dfs,insurer, rows, columns):
    # Create a new dataframe with the first column as "Nombre de la Cuenta"
    new_df = pd.DataFrame(rows, columns=["Nombre de la Cuenta"])

    # Add new columns to the dataframe

    new_df[insurer] = None

    # Iterate over each dataframe
    for df in dfs:
        # Iterate over each row
        for i, row in enumerate(rows):
            # Iterate over each column
            for column in columns:
                # Get the value at the intersection of the row and column
                value = get_value_from_df(df, row, column)

                # If the value is not an error message, assign it to the new dataframe
                if not isinstance(value, str):
                    new_df.at[i, insurer] = value

    # Return the new dataframe
    return new_df

In [68]:
def create_dict_from_df(df):
    # Initialize an empty dictionary
    new_dict = {}

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Use the 'Nombre de la Cuenta' value as the key and the 'El Comercio Paraguayo S.A. De Seguros' value as the value
        new_dict[row['Nombre de la Cuenta']] = row['El Comercio Paraguayo S.A. De Seguros']

    # Return the new dictionary
    return new_dict

In [74]:
def create_dict_from_df2(df):
    # Initialize an empty dictionary
    new_dict = {}

    # Get the second key from the old dictionary
    second_key = list(df.columns)[1]

    # Set the first key-value pair
    new_dict["Insurer"] = second_key

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Use the 'Nombre de la Cuenta' value as the key and the 'El Comercio Paraguayo S.A. De Seguros' value as the value
        new_dict[row['Nombre de la Cuenta']] = row[second_key]

    # Return the new dictionary
    return new_dict

In [75]:
container = create_new_dfs(dataframes,'El Comercio Paraguayo S.A. De Seguros', rows, columns)

In [ ]:
print(container.keys()) 
print(container)

In [81]:
new_dict = create_dict_from_df2(container)
# print(new_dict.keys())
print(type(new_dict))

<class 'dict'>


In [92]:
def insert_data(data_dict, collection_name):
    uri = "MONGODB_URI"
    # Create a new client and connect to the server
    client = MongoClient(uri, server_api=ServerApi('1'))    

    # Access the 'insurance_market_analisis' database
    db = client['insurance_market_statistics']

    # Convert numpy.int64 to int
    for key, value in data_dict.items():
        if isinstance(value, np.int64):
            data_dict[key] = int(value)

    # Access the specified collection
    collection = db[collection_name]

    # Insert the data
    collection.insert_one(data_dict)

In [93]:
for insurer in columns:
    container = create_new_dfs(dataframes, insurer, rows, columns)
    new_dict = create_dict_from_df2(container)
    insert_data(new_dict, "insurance_companies")
    print(new_dict)
    

{'Insurer': 'El Comercio Paraguayo S.A. De Seguros', 'Total de Activos': 5907591487366, 'Total de Pasivos': 3458736792972, 'Capital Social': 1444211080763, 'Resultado del Ejercicio': 244190177799, 'Total Patrimonio Neto': 2448854694394, 'Primas Directas': 2218164421033.0, 'Primas Reaseguros Aceptados - Local': 15565099667, 'Siniestros Seguros Directos': 778719598639, 'Resultado Técnico Bruto [7]=[3]-[6]': 963020974727, 'Gastos De Producción': 431669724235, 'Gastos De Cesión Reaseguros - Local': 258969296, 'Gastos De Cesión Reaseguros - Exterior': 49104781164, 'Gastos Técnicos De Explotación': 546964481753, 'Constitución De Previsiones': 51882958558, 'Resultado Técnico Neto [11]=[7]+[10]': 125331048705, 'Resultado Total del Ejercicio': 244190177799, '_id': ObjectId('65e768b1da652931e0e910f5')}
{'Insurer': 'La Rural S.A. De Seguros', 'Total de Activos': 5907591487366, 'Total de Pasivos': 3458736792972, 'Capital Social': 1444211080763, 'Resultado del Ejercicio': 244190177799, 'Total Patri